  # **AKADEMI EDUCATION**
# **Première cohorte (2025): Science des données et intelligence artificielle**
#### *Phase 5: PROJET DE SCIENCE DES DONNÉES*

**Noms des étudiants du groupe: Riché FLEURINORD et Micka LOUIS**   
**Rythme d’apprentissage: Autonome**  
**Date de soutenance: 27 octobre 2025**  
**Noms des instructeurs: Wedter JEROME et Geovany Batista Polo LAGUERRE**  
**Lien de l’article de blog (lien du dépôt GitHub): https://github.com/richefleuriord/Bank_Customer_Churn_Prediction.git**

# *4-Modélisation*

## Introduction et Objectifs de la Modélisation

### Objectif général
L’objectif principal de cette section est de **construire, évaluer et comparer plusieurs modèles de Machine Learning** afin de prédire la probabilité qu’un client quitte la banque (*churn*).  
Après avoir soigneusement préparé les données dans la section précédente, cette phase vise à **exploiter les variables sélectionnées et transformées** pour générer un modèle fiable, interprétable et performant.

### Objectifs spécifiques
- Développer une approche systématique de modélisation à l’aide de **pipelines** pour automatiser les étapes de prétraitement et d’apprentissage.  
- Utiliser **GridSearchCV** afin d’optimiser les hyperparamètres de chaque algorithme.  
- Évaluer les performances à l’aide d’indicateurs pertinents tels que **Accuracy**, **ROC-AUC**, **Rapport de classification** et **Matrice de confusion**.  
- Comparer différents modèles (régression logistique, forêts aléatoires, XGBoost et réseau de neurones) pour identifier celui qui offre le meilleur compromis entre performance et complexité.

### Méthodologie
1. **Chargement des données préparées** : utilisation des jeux de données `train`, `validation` et `test` créés à l’étape précédente.  
2. **Construction de pipelines** : intégration des transformations (scaling, modèle, etc.) dans un processus automatisé.  
3. **Recherche d’hyperparamètres optimaux** via `GridSearchCV` avec validation croisée.  
4. **Évaluation sur le jeu de validation** pour comparer les performances entre modèles.  
5. **Test final** du meilleur modèle sélectionné sur le jeu de test indépendant.  
6. **Sauvegarde du modèle final** pour une utilisation ultérieure dans la phase de déploiement.

### Modèles étudiés
- **Régression Logistique**: modèle linéaire de base pour l’interprétation et la compréhension des facteurs clés.  
- **Random Forest**: méthode d’ensemble basée sur plusieurs arbres de décision.  
- **XGBoost**: algorithme de boosting puissant, efficace pour la classification déséquilibrée.  
- **Réseau de Neurones (MLP)**: approche non linéaire capable de capturer des relations complexes entre les variables.

---

> **En résumé:**  
> Cette section vise à déterminer **le modèle le plus performant et le plus robuste** pour la prédiction du *churn*, en combinant rigueur méthodologique, optimisation des paramètres et analyse critique des résultats.


In [1]:
# ============================================================
# IMPORT DES LIBRAIRIES
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import joblib
import os

# Création du dossier "Models" 
os.makedirs("Models", exist_ok=True)

# ============================================================
# CHARGEMENT DES DONNÉES PRÉPARÉES
# ============================================================

X_train = pd.read_csv("../Data/X_train_prepared.csv")
y_train = pd.read_csv("../Data/y_train_prepared.csv").values.ravel()

X_val = pd.read_csv("../Data/X_val_prepared.csv")
y_val = pd.read_csv("../Data/y_val_prepared.csv").values.ravel()

X_test = pd.read_csv("../Data/X_test_prepared.csv")
y_test = pd.read_csv("../Data/y_test_prepared.csv").values.ravel()

print("Données chargées avec succès !")

# ============================================================
# DÉFINITION DES MODÈLES ET PIPELINES
# ============================================================

pipelines = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("model", RandomForestClassifier(random_state=42))
    ]),
    "XGBoost": Pipeline([
        ("model", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
    ]),
    "NeuralNetwork": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPClassifier(max_iter=300, random_state=42))
    ])
}

# ============================================================
# DÉFINITION DES GRILLES DE PARAMÈTRES POUR GRIDSEARCHCV
# ============================================================

param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1, 10],
        "model__penalty": ["l2"],
        "model__solver": ["lbfgs", "liblinear"]
    },
    "RandomForest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [5, 10, 20],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [3, 5, 7],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__subsample": [0.8, 1.0]
    },
    "NeuralNetwork": {
        "model__hidden_layer_sizes": [(50,), (100,), (100, 50)],
        "model__activation": ["relu", "tanh"],
        "model__alpha": [0.0001, 0.001],
        "model__learning_rate_init": [0.001, 0.01]
    }
}

# ============================================================
# GRIDSEARCHCV SUR CHAQUE MODÈLE
# ============================================================

best_models = {}
results = []

for name, pipeline in pipelines.items():
    print(f"\n Entraînement du modèle : {name}")
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[name],
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_models[name] = best_model

    print(f"Meilleurs paramètres {name} :", grid.best_params_)
    print(f"Score ROC-AUC (train) : {grid.best_score_:.4f}")

    # Évaluation sur le jeu de validation
    y_val_pred = best_model.predict(X_val)
    val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
    val_acc = accuracy_score(y_val, y_val_pred)

    results.append({
        "Model": name,
        "BestParams": grid.best_params_,
        "Val Accuracy": val_acc,
        "Val ROC-AUC": val_auc
    })

# Résumé des résultats
results_df = pd.DataFrame(results)
print("\n Résumé des performances sur le jeu de validation :")
print(results_df)

# ============================================================
# ÉVALUATION DU MEILLEUR MODÈLE SUR LE TEST SET
# ============================================================

best_model_name = results_df.sort_values(by="Val ROC-AUC", ascending=False).iloc[0]["Model"]
final_model = best_models[best_model_name]

print(f"\n Meilleur modèle sélectionné : {best_model_name}")

y_test_pred = final_model.predict(X_test)
y_test_proba = final_model.predict_proba(X_test)[:, 1]

print("\n Évaluation finale sur le jeu de test :")
print(classification_report(y_test, y_test_pred))
print("Accuracy :", accuracy_score(y_test, y_test_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_test_proba))
print("Confusion Matrix :\n", confusion_matrix(y_test, y_test_pred))

# ============================================================
# SAUVEGARDE DU MEILLEUR MODÈLE
# ============================================================

joblib.dump(final_model, f"Models/Best_Model_{best_model_name}.pkl")
print(f"\n Modèle sauvegardé : Best_Model_{best_model_name}.pkl")


Données chargées avec succès !

 Entraînement du modèle : LogisticRegression
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Meilleurs paramètres LogisticRegression : {'model__C': 0.1, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
Score ROC-AUC (train) : 0.9095

 Entraînement du modèle : RandomForest
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.3min finished


Meilleurs paramètres RandomForest : {'model__max_depth': 20, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Score ROC-AUC (train) : 0.9563

 Entraînement du modèle : XGBoost


...

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   48.7s finished
C:\Users\HP\anaconda3\envs\learn-env\lib\site-packages\xgboost\core.py:158: UserWarning: [14:07:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Meilleurs paramètres XGBoost : {'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__n_estimators': 200, 'model__subsample': 0.8}
Score ROC-AUC (train) : 0.9590

 Entraînement du modèle : NeuralNetwork
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 11.5min finished
C:\Users\HP\anaconda3\envs\learn-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Meilleurs paramètres NeuralNetwork : {'model__activation': 'relu', 'model__alpha': 0.0001, 'model__hidden_layer_sizes': (50,), 'model__learning_rate_init': 0.001}
Score ROC-AUC (train) : 0.9371

 Résumé des performances sur le jeu de validation :
                Model                                         BestParams  \
0  LogisticRegression  {'model__C': 0.1, 'model__penalty': 'l2', 'mod...   
1        RandomForest  {'model__max_depth': 20, 'model__min_samples_l...   
2             XGBoost  {'model__learning_rate': 0.1, 'model__max_dept...   
3       NeuralNetwork  {'model__activation': 'relu', 'model__alpha': ...   

   Val Accuracy  Val ROC-AUC  
0      0.782000     0.725162  
1      0.844667     0.848006  
2      0.851333     0.856335  
3      0.810667     0.806448  

 Meilleur modèle sélectionné : XGBoost

 Évaluation finale sur le jeu de test :
              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1195
           1       0.70    